In [ ]:
import csv
import operator
import datetime
import os

from trainstop import *
from locations import *
from dateconversion import *

In [ ]:
#import data from all files
data_by_date = {}

for filename in os.listdir('./data'):
    date = datetime.date(int(filename[0:4]),int(filename[5:7]),int(filename[8:10]))
    file = open("data/" + date.isoformat() + "_IstDaten.csv")
    data_by_date[date] = csv.DictReader(file, delimiter=';')

#import data from files
train_stops = []
    
for date in data_by_date:
    for train in data_by_date[date]:
        train_stops.append(create_train_stop(train))


In [ ]:
#import data from one file
file = open("data/2016-10-12_IstDaten.csv")
reader = csv.DictReader(file,delimiter=';')

train_stops = []
for train in reader:
    train_stops.append(TrainStop(train))

In [ ]:
#count delay per stop

train_stops_delay = {}

for train_stop in train_stops:
    if train_stop not in train_stops_delay:
        train_stops_delay[train_stop.stop_name] = 0
    train_stops_delay[train_stop.stop_name] += train_stop.arrival_delay()
    
    

In [ ]:
train_stops_delay

In [ ]:
sorted_by_delay = sorted(train_stops_delay.items(), key=operator.itemgetter(1))
sorted_by_delay

In [ ]:
train_stops_delay["Genève"]

In [ ]:
station_names = []
for station_name in train_stops_delay:
    station_names.append(station_name)

save_to_file(get_locations(station_names))

In [3]:
from mpl_toolkits.basemap import Basemap, shiftgrid, cm
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset

# read in etopo5 topography/bathymetry.
url = 'http://ferret.pmel.noaa.gov/thredds/dodsC/data/PMEL/etopo5.nc'
etopodata = Dataset(url)

topoin = etopodata.variables['ROSE'][:]
lons = etopodata.variables['ETOPO05_X'][:]
lats = etopodata.variables['ETOPO05_Y'][:]
# shift data so lons go from -180 to 180 instead of 20 to 380.
topoin,lons = shiftgrid(180.,topoin,lons,start=False)

# plot topography/bathymetry as an image.

# create the figure and axes instances.
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
# setup of basemap ('lcc' = lambert conformal conic).
# use major and minor sphere radii from WGS84 ellipsoid.
m = Basemap(llcrnrlon=-145.5,llcrnrlat=1.,urcrnrlon=-2.566,urcrnrlat=46.352,\
            rsphere=(6378137.00,6356752.3142),\
            resolution='l',area_thresh=1000.,projection='lcc',\
            lat_1=50.,lon_0=-107.,ax=ax)
# transform to nx x ny regularly spaced 5km native projection grid
nx = int((m.xmax-m.xmin)/5000.)+1; ny = int((m.ymax-m.ymin)/5000.)+1
topodat = m.transform_scalar(topoin,lons,lats,nx,ny)
# plot image over map with imshow.
im = m.imshow(topodat,cm.GMT_haxby)
# draw coastlines and political boundaries.
m.drawcoastlines()
m.drawcountries()
m.drawstates()
# draw parallels and meridians.
# label on left and bottom of map.
parallels = np.arange(0.,80,20.)
m.drawparallels(parallels,labels=[1,0,0,1])
meridians = np.arange(10.,360.,30.)
m.drawmeridians(meridians,labels=[1,0,0,1])
# add colorbar
cb = m.colorbar(im,"right", size="5%", pad='2%')
ax.set_title('ETOPO5 Topography - Lambert Conformal Conic')
plt.show()

ImportError: No module named 'mpl_toolkits.basemap'